In [1]:
import pandas as pd
import numpy as np
import datetime
np.set_printoptions(suppress=True)
pd.set_option('display.max_rows', 20)

In [4]:
from yahoo_fin.stock_info import get_data
Top10CC = ['BTC-USD', 'ETH-USD', 'USDT-USD', 'XRP-USD', 'BCH-USD', 'LINK-USD','BNB-USD',  'ADA-USD', 'LTC-USD',
           'EOS-USD']
hisPrices = {}
for cc in Top10CC:
    hisPrices[cc] = get_data(cc, '1/1/2019', datetime.date.today(),True, '1wk')
    
Prices = pd.DataFrame(index=np.arange(hisPrices['BTC-USD'].shape[0]),columns = Top10CC)
Prices.set_index(hisPrices['BTC-USD'].index, inplace = True)
for x in hisPrices:
    Prices[x] = hisPrices[x].adjclose
hisPrices.clear

<function dict.clear>

In [5]:
returns = np.log(Prices/Prices.shift(1)).fillna(value=0) #log(R_t/R_t-1)

In [6]:
returns.mean()

BTC-USD     0.014296
ETH-USD     0.012116
USDT-USD   -0.000125
XRP-USD     0.004129
BCH-USD     0.004976
LINK-USD    0.034748
BNB-USD     0.014863
ADA-USD     0.010786
LTC-USD     0.006076
EOS-USD     0.000352
dtype: float64

In [7]:
import tushare as ts
ts.set_token('d22a3bc6484b9fd9618e97dfd6249e49d7146267e1167bdc375de7c1')
pro = ts.pro_api()
df = pro.libor(curr_type='USD', start_date='20190101', end_date='20201001')

In [8]:
def findLIBOR(date):
    for i in df.index:
        if date == df.loc[i]['date']:
            return df.loc[i]['1w']
            break
    return 0

In [9]:
LIBOR = returns.copy(deep =True).drop(columns = Top10CC)
LIBOR['LIBOR']=0.0
for i in LIBOR.index:
    d = str(i).split(" ")[0].split("-")
    date = d[0]+d[1]+d[2]
    LIBOR.loc[i]['LIBOR'] = findLIBOR(date)
# change 0.0 to the adjacent value
for i in range(1,len(LIBOR)):
    if LIBOR.iloc[i]['LIBOR'] == 0.0:
        LIBOR.iloc[i]['LIBOR'] = LIBOR.iloc[i-1]['LIBOR']

In [10]:
LIBOR

,LIBOR
2019-01-01,0.00000
2019-01-08,2.40875
2019-01-15,2.40200
2019-01-22,2.40888
2019-01-29,2.41388
...,...
2020-11-03,0.10563
2020-11-10,0.10563
2020-11-17,0.10563
2020-11-24,0.10563


In [11]:
def getMean(returns,start,end):
    return returns.iloc[start:end].mean()
def getCov(returns,start,end):
    return returns.iloc[start:end].cov()

In [12]:
getCov(returns, 0 , len(returns))

,BTC-USD,ETH-USD,USDT-USD,XRP-USD,BCH-USD,LINK-USD,BNB-USD,ADA-USD,LTC-USD,EOS-USD
BTC-USD,0.009344,0.010098,0.000062,0.006929,0.010513,0.008547,0.007649,0.008801,0.009374,0.009151
ETH-USD,0.010098,0.015900,0.000033,0.011846,0.013699,0.014177,0.010983,0.013783,0.013040,0.012932
USDT-USD,0.000062,0.000033,0.000020,0.000016,0.000015,0.000112,-0.000034,-0.000023,0.000041,0.000027
XRP-USD,0.006929,0.011846,0.000016,0.014217,0.010638,0.010449,0.008275,0.011509,0.010219,0.010417
BCH-USD,0.010513,0.013699,0.000015,0.010638,0.017984,0.010505,0.009846,0.013222,0.013758,0.014044
LINK-USD,0.008547,0.014177,0.000112,0.010449,0.010505,0.035093,0.012087,0.013123,0.010684,0.011101
BNB-USD,0.007649,0.010983,-0.000034,0.008275,0.009846,0.012087,0.015497,0.011240,0.010326,0.010416
ADA-USD,0.008801,0.013783,-0.000023,0.011509,0.013222,0.013123,0.011240,0.017902,0.012392,0.012973
LTC-USD,0.009374,0.013040,0.000041,0.010219,0.013758,0.010684,0.010326,0.012392,0.015293,0.012999
EOS-USD,0.009151,0.012932,0.000027,0.010417,0.014044,0.011101,0.010416,0.012973,0.012999,0.014626


In [13]:
returns.mean()

BTC-USD     0.014296
ETH-USD     0.012116
USDT-USD   -0.000125
XRP-USD     0.004129
BCH-USD     0.004976
LINK-USD    0.034748
BNB-USD     0.014863
ADA-USD     0.010786
LTC-USD     0.006076
EOS-USD     0.000352
dtype: float64

In [14]:
def getRisk_free(LIBOR,start,end):
    return LIBOR.iloc[start:end]['LIBOR'].mean()/5200

In [15]:
from functools import reduce
def std(w,cov):
    return np.sqrt(reduce(np.dot,[w,cov,w.T]))

In [16]:
import scipy.optimize as solver
def getOptimalPortfolio(returns, LIBOR,start,end):
    
    #general initialization
    mean = getMean(returns,start,end)
    cov = getCov(returns,start,end)
    risk_free =  getRisk_free(LIBOR,start,end)
    max_sharpe = 0.0
    weights=[]
    
    #Initialization of solver.minimize
    return_change_range = np.arange(min(mean),max(mean),(max(mean)-min(mean))/100)
    w0 = np.array([1/len(Top10CC) for x in range(len(Top10CC))]) #equal weights
    #bounds = tuple((0,1) for x in range(len(Top10CC))) #boundary of each weight. (0,1)means no short selling
    
    for i in return_change_range:
        constraints = [{'type':'eq','fun': lambda x: sum(x) - 1.0}, {'type':'eq', 'fun': lambda x: sum(x*mean) - i}]
        outcome = solver.minimize(std, args=(cov),x0=w0,constraints = constraints) #args is passed to std()
        if (i-risk_free)/outcome.fun > max_sharpe:
            max_sharpe = (i-risk_free)/outcome.fun
            weights.append(outcome.x)
            
    optimalReturn = sum(weights[-1]*mean)
    optimalWeights = weights[-1]
    
    return [max_sharpe, optimalReturn, optimalWeights]

In [17]:
# closed form

In [18]:
getOptimalPortfolio(returns, LIBOR,0,len(returns))

[0.30320575507911574,
 0.034747576269990366,
 array([ 1.24584408, -0.12156806,  0.35469281, -0.06807025, -0.09247962,
         0.34702055,  0.28103091,  0.29366948,  0.03999149, -1.28013139])]

In [19]:
# x = pd.DataFrame(returns.iloc[11]).stack().unstack(0)

In [20]:
CAPM_data = pd.DataFrame()

Thestarttime=datetime.datetime.now()
for i in range(25, len(returns) - 1):
    starttime = datetime.datetime.now()
    
    # data of the first i weeks
    truncated_returns_CC = returns.iloc[0:i]
    # data of the following week (i+1)th week
    real_returns_CC = returns.iloc[i+1]
    
    #Calculating the optimal portfolio using data of the first i weeks
    [sharpeRatio, expected_returns_portfolio, weights] = getOptimalPortfolio(truncated_returns_CC, LIBOR,0,len(truncated_returns_CC))   
    real_returns_portfolio = sum(weights*real_returns_CC)

    # cov of 1 week is none
    #realized_returns  = getOptimalPortfolio(real_returns_CC,LIBOR,0,len(real_returns_CC))[1]
    
    Std = std(weights, truncated_returns_CC.cov()) 
    CAPM_data = CAPM_data.append(pd.Series({'Expected Returns' : expected_returns_portfolio, 
                                            'Real Returns' : real_returns_portfolio, 
                                            'Sharpe Ratio' : sharpeRatio , 'Std' : Std}, name = returns.iloc[i+1].name))   
    
    endtime = datetime.datetime.now()
    print (str(returns.iloc[i+1].name)+ "  Time cost: "+str((endtime - starttime).seconds) +" seconds")

Theendtime = datetime.datetime.now()
print ("Total time cost: " +str((Theendtime - Thestarttime).seconds)+" seconds")

KeyboardInterrupt: 

In [ ]:
# whether can short sell

In [ ]:
# transaction cost

In [ ]:
CAPM_data

In [21]:
weights = np.array([1/len(Top10CC) for x in range(len(Top10CC))])
equal_weights_data = pd.DataFrame()

for i in range(25, len(returns) - 1):
    
    truncated_returns_CC = returns.iloc[0:i]
    real_returns_CC = returns.iloc[i+1]
    
    real_returns_portfolio = sum(weights*real_returns_CC)
    expected_returns_portfolio = sum(weights*truncated_returns_CC.mean())
    
    Std = std(weights, truncated_returns_CC.cov()) 
    risk_free =  getRisk_free(LIBOR,0,len(truncated_returns_CC))
    sharpeRatio = ((expected_returns_portfolio - risk_free)/Std)
    
    #realized_returns  = getOptimalPortfolio(real_returns_CC,LIBOR,0,len(real_returns_CC))[1]
    
    equal_weights_data = equal_weights_data.append(pd.Series({'Expected Returns' : expected_returns_portfolio, 
                                                              'Real Returns' : real_returns_portfolio, 
                                                              'Sharpe Ratio' : sharpeRatio, 
                                                              'Std' : Std}, name = returns.iloc[i+1].name)) 


In [23]:
equal_weights_data

,Expected Returns,Real Returns,Sharpe Ratio,Std
2019-07-01,0.037490,0.011483,0.438409,0.084500
2019-07-08,0.034962,-0.224941,0.411944,0.083790
2019-07-15,0.034092,-0.014632,0.408895,0.082287
2019-07-22,0.024841,-0.041803,0.258346,0.094429
2019-07-29,0.023480,0.073238,0.247631,0.093016
...,...,...,...,...
2020-11-03,0.008814,0.077307,0.088837,0.096191
2020-11-10,0.008206,0.040538,0.082815,0.095869
2020-11-17,0.008918,0.256984,0.090502,0.095626
2020-11-24,0.009241,-0.130978,0.094337,0.095186


In [22]:
returns.iloc[1].name

Timestamp('2019-01-08 00:00:00')